In [ ]:
%matplotlib inline
import yt
import logging
logging.getLogger('yt').setLevel(logging.ERROR)
import numpy as np
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 150
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
sys.path.append('/home/ychen/lib/util')
import pickle

import util

In [ ]:
#dir = '/home/ychen/d9/FLASH4/stampede/0529_L45_M10_b1_h1/'

def rescan(dir, printlist=False):
    files = util.scan_files(dir, '*hdf5_plt_cnt_[0-9][0-9][0-9][0-9]', \
                             printlist=printlist, walk=True)
    return files
#files = rescan(dir, True)


In [ ]:
def get_Bs(fn):
    #print fn
    try:
        ds = yt.load(fn)
        ad = ds.all_data()
        Btor = sum(ad['magnetic_field_toroidal']**2*ad['cell_volume'])/(8.0*np.pi)
        Bpol = sum(ad['magnetic_field_poloidal']**2*ad['cell_volume'])/(8.0*np.pi)
        Emag = sum(ad['magnetic_pressure']*ad['cell_volume'])
        t = ds.current_time
        return t.in_units('s'), Btor.in_units('erg'), \
               Bpol.in_units('erg'), Emag.in_units('erg')
    except:
        print 'Error!'
        raise Exception

def get_Bflux(fn):
    try:
        ds = yt.load(fn)
        ad = ds.all_data()
        Btor_flux = sum(ad['magnetic_field_toroidal']*(ad['dx']*ad['dy']))
        Bx_flux = sum(ad['magnetic_field_x']*ad['dy']*ad['dz'])
        By_flux = sum(ad['magnetic_field_y']*ad['dx']*ad['dz'])
        Bz_flux = sum(ad['magnetic_field_z']*ad['dx']*ad['dy'])
        t = ds.current_time
        return t.in_units('s'), Btor_flux.in_units('gauss*cm**2'),\
               Bx_flux.in_units('gauss*cm**2'), By_flux.in_units('gauss*cm**2'), Bz_flux.in_units('gauss*cm**2')
    except:
        print 'Error!'
        raise Exception
        
def get_mags(func, dirname, parallel=True):
    files = rescan(dirname, True)
    #ts = []
    #Btors, Bpols, Emags = [], [], []
    fnames = [f.fullpath for f in files[1:-1:5]]
    print [fn.split('_')[-1] for fn in fnames]
    if parallel:
        pool = multiprocessing.Pool(2)
    #print fnames
        result = pool.map(func, fnames)
    else:
        result = map(func, fnames)
    return result

    #ts.append(t)
    #Btors.append(Btor)
    #Bpols.append(Bpol)
    #Emags.append(Emag)

In [ ]:
result = get_mags(get_Bflux, '/home/ychen/d9/FLASH4/stampede/0529_L45_M10_b1_h1/')

In [ ]:
print result

In [ ]:
items = zip(*result)
ts = items[0]
#labels = ['toroidal B', 'poloidal_B', 'B energy']
labels = ['Btor_flux', 'Bx_flux', 'By_flux', 'Bz_flux']
for item, label in zip(items[1:], labels):
    plt.plot(np.array(ts)/3.154E13, np.array(item), label=label)

#plt.plot(ts/3.154E13, (np.array(Emags)-np.array(Emags[0]))*30, label='magnetic E')
plt.legend(loc=0)
plt.xlim(0,25)
plt.xlabel('t (Myr)')
plt.ylabel('Bflux (gauss*cm**2)')
#plt.title('h1')

In [ ]:
dirnames = ['/d/d9/ychen/FLASH4/stampede/0529_L45_M10_b1_h1',\
            '/d/d8/ychen/MHD_Jet/0605_L45_M10_b1_h0',\
            '/d/d8/ychen/MHD_Jet/0605_L45_M10_b1_hinf']
table = []
import time
t0 = time.time()
for dirname in dirnames:
    print dirname
    table.append(get_mags(get_Bflux, dirname))
    print 'Elapsed time:', time.time() - t0
    t0 = time.time()

In [ ]:
#table = [
[(5.00158718802e+12, 1.6727489611e+56, 2.42682918753e+55, 2.251799796e+56),\
(5.50020360592e+13, 3.24285650908e+56, 2.21167405584e+56, 8.30032216414e+56),\
(1.0500302562e+14, 4.54888523887e+56, 3.87764143691e+56, 1.26738050557e+57),\
(1.55003168749e+14, 5.80657059551e+56, 4.79668108846e+56, 1.60216021358e+57),\
(2.05000949569e+14, 7.11416410166e+56, 6.6590198766e+56, 2.06130668221e+57),\
(2.55001272331e+14, 7.62939951274e+56, 6.97990190138e+56, 2.19740266076e+57),\
(3.0500296661e+14, 9.11058723553e+56, 7.46287172779e+56, 2.51473881609e+57),\
(3.55001703146e+14, 9.15338677004e+56, 7.6672754075e+56, 2.51043735755e+57),\
(4.05002067412e+14, 9.43238472673e+56, 8.19263119253e+56, 2.72400021638e+57),\
(4.55003040994e+14, 9.51928684204e+56, 7.88473176088e+56, 2.71359026867e+57),\
(5.05000740072e+14, 1.00562488923e+57, 8.73688614549e+56, 2.92272507291e+57),\
(5.55003862373e+14, 1.05514645061e+57, 8.44041901706e+56, 2.96989922472e+57),\
(6.05003530679e+14, 1.13531573848e+57, 8.88705110467e+56, 3.15731070027e+57),\
(6.55001266148e+14, 1.10580642453e+57, 9.47659641619e+56, 3.1871690704e+57),\
(7.05001446833e+14, 1.16508760741e+57, 9.7964696521e+56, 3.40251629155e+57)]
                                                                                                                                                                                                                                             
[(5.00375290685e+12, 6.00583277122e+53, 2.2285979346e+55, 6.8565913686e+55),                                                                                                                                                                  
(5.5001721206e+13, 5.04547169651e+55, 5.57593395936e+55, 2.48357522616e+56),                                                                                                                                                                 
(1.05002204722e+14, 9.20566902457e+55, 8.53241620988e+55, 3.57894917569e+56),                                                                                                                                                                
(1.55001408635e+14, 1.41864697311e+56, 1.38223519892e+56, 5.3744921759e+56),                                                                                                                                                                 
(2.05000952714e+14, 1.7624737866e+56, 1.66094466291e+56, 6.374613925e+56),                                                                                                                                                                   
(2.5500119781e+14, 2.21992752527e+56, 2.09497084574e+56, 7.95568586527e+56),                                                                                                                                                                 
(3.05000666278e+14, 2.64036785444e+56, 2.5004326583e+56, 9.1615263261e+56),                                                                                                                                                                  
(3.55003711979e+14, 3.05979915231e+56, 2.93841822736e+56, 1.04950700735e+57),                                                                                                                                                                
(4.05002623171e+14, 3.13185796958e+56, 2.9760650682e+56, 1.09043890806e+57),                                                                                                                                                                 
(4.55002874305e+14, 3.44265990688e+56, 3.12635664683e+56, 1.15301780285e+57),                                                                                                                                                                
(5.05000255108e+14, 3.60536094934e+56, 3.32740832137e+56, 1.20810410715e+57)]                                                                                                                                                                
                                                                                                                                                                                                                                             
[(5.00041537583e+12, 2.20412665505e+56, 2.27539502142e+55, 2.56228673825e+56),                                                                                                                                                                
(5.50015347484e+13, 2.74849603093e+56, 1.94320567073e+56, 6.25131381146e+56),                                                                                                                                                                
(1.05001916822e+14, 3.88993979349e+56, 2.87918839273e+56, 9.48014021153e+56),                                                                                                                                                                
(1.55002086685e+14, 4.2586812795e+56, 3.35115392977e+56, 1.08290178477e+57),                                                                                                                                                                 
(2.05001863893e+14, 4.9482428017e+56, 4.18110344257e+56, 1.30793191173e+57),                                                                                                                                                                 
(2.55000087634e+14, 5.21776751881e+56, 4.7479761541e+56, 1.42643892774e+57),                                                                                                                                                                 
(3.05002418053e+14, 6.05313097336e+56, 5.27057954215e+56, 1.64458400936e+57),                                                                                                                                                                
(3.55002129376e+14, 6.37837135398e+56, 5.92916312747e+56, 1.8189919104e+57),                                                                                                                                                                 
(4.05000581613e+14, 7.42611691071e+56, 6.13973768585e+56, 2.01770867138e+57),                                                                                                                                                                
(4.55002496761e+14, 7.29045238634e+56, 6.44809167787e+56, 2.04528912681e+57)]]                                                                                                                                                                

In [ ]:
import time
picklename = time.strftime("Bflux_table_20160317_114823.pickle")
pickle.dump(table, open( picklename, "wb" ))
#for item in table:
#    print 
#    for i in item:
#        print i

In [ ]:
fig = plt.figure(figsize=(8,6))
names = [dirname.split('/')[-1].split('_')[-1] for dirname in dirnames]
colors = ['r', 'b', 'g', 'cyan']

labels = ['Btor_flux', 'Bx_flux', 'By_flux', 'Bz_flux']
lss = ['-', ':', '--', '-.']

for label, ls in zip(labels, lss):
    plt.plot([0.0], [0.0], ls=ls, c='k', label=label)
for i in range(len(table)):
    items = zip(*table[i])
    ts = items[0]
    #labels = ['toroidal B', 'poloidal_B', 'B energy']
    first = True
    for item, label, ls in zip(items[1:], labels, lss):
        if first:
            plt.plot(np.array(ts)/3.154E13, np.array(item), c=colors[i], ls=ls, label=names[i])
            first = False
        else:
            plt.plot(np.array(ts)/3.154E13, np.array(item), c=colors[i], ls=ls)

plt.legend(loc=2, ncol=2)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(2.563E62, 2.563E62+9E59)
plt.ylim(-0.5E42, 2.5E42)
plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('B flux (gauss*cm**2)')
plt.xlabel('t (Myr)')
plt.show()

In [ ]:
names = [dirname.split('/')[-1].split('_')[-1] for dirname in dirnames]
colors = ['r', 'b', 'g', 'cyan']
for i in range(len(table)):
    ts, Btors, Bpols, Emags = zip(*(table[i]))
    ts = np.array(ts)
    plt.plot(ts/3.154E13, np.array(Btors)/np.array(Bpols), c=colors[i], label=names[i])
    plt.plot(ts/3.154E13, np.array(Btors)/np.array(Emags), c=colors[i], ls=':')
    #plt.plot(ts/3.154E13, Bpols, c=colors[i], ls=':')
    #plt.plot(ts/3.154E13, Emags, c=colors[i], ls='-', label=names[i])
plt.plot([0.0], [0.0], ls=':', c='r', label='E_Btor/E_mag')
#plt.plot([0.0], [0.0], ls=':', c='r', label='E_Bpol')

plt.legend(loc=1)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(2.563E62, 2.563E62+9E59)
plt.ylim(0, 2)
plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('E_Btor/E_Bpol')
plt.xlabel('t (Myr)')
plt.show()

In [ ]:
with open('Bflux_table_0.7_1kpc_disk.pickle', 'r') as f:
    results = pickle.load(f)

for key in results.keys():
    print key
    print len(results[key])

In [ ]:
fig = plt.figure(figsize=(8,6))
names = [dirname.split('/')[-1][6:] for dirname in results.keys()]
print names
colors = {}

for dirname in results.keys():
    if 'h1' in dirname:
        colors[dirname] = 'r'
    elif 'h0' in dirname:
        colors[dirname] = 'b'
    elif 'hinf' in dirname:
        colors[dirname] = 'g'
#colors = ['r', 'b', 'g', 'cyan']

labels = ['Btor_flux', 'Bx_flux', 'By_flux', 'Bz_flux']
lss = ['-', '--', '-.', ':']

for label, ls in zip(labels, lss):
    plt.plot([0.0], [0.0], ls=ls, c='k', label=label)
i=0
for dirname in results.keys():
    items = zip(*sorted(results[dirname]))
    ts = items[0]
    #labels = ['toroidal B', 'poloidal_B', 'B energy']
    first = True
    for item, label, ls in zip(items[1:], labels, lss):
        if first:
            plt.plot(np.array(ts)/3.154E13, np.array(item), c=colors[dirname], marker='x', ms=2,
                     ls=ls, label=dirname.split('/')[-1][11:])
            first = False
        else:
            plt.plot(np.array(ts)/3.154E13, np.array(item), c=colors[dirname], ls=ls)

plt.legend(loc=1, ncol=2)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(2.563E62, 2.563E62+9E59)
plt.ylim(-0.01E42, 0.12E42)
plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('B flux (gauss*cm**2)')
plt.xlabel('t (Myr)')
plt.title('B flux within 0.7x1 kpc disk')
plt.show()